In [0]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler, VectorSlicer, PCA
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.functions import vector_to_array
from pyspark.sql.functions import col, lit
from pyspark.ml.clustering import KMeans
import re
from functools import reduce

In [0]:
train_df = spark.read.csv('/FileStore/tables/train_features.csv', header=True, inferSchema=True)
target_df = spark.read.csv('/FileStore/tables/train_targets_scored.csv', header=True, inferSchema=True)
test_df = spark.read.csv('/FileStore/tables/test_features.csv', header=True, inferSchema=True)

In [0]:
train_df.cache()
target_df.cache()
test_df.cache()

Out[14]: DataFrame[sig_id: string, cp_type: string, cp_time: int, cp_dose: string, g-0: double, g-1: double, g-2: double, g-3: double, g-4: double, g-5: double, g-6: double, g-7: double, g-8: double, g-9: double, g-10: double, g-11: double, g-12: double, g-13: double, g-14: double, g-15: double, g-16: double, g-17: double, g-18: double, g-19: double, g-20: double, g-21: double, g-22: double, g-23: double, g-24: double, g-25: double, g-26: double, g-27: double, g-28: double, g-29: double, g-30: double, g-31: double, g-32: double, g-33: double, g-34: double, g-35: double, g-36: double, g-37: double, g-38: double, g-39: double, g-40: double, g-41: double, g-42: double, g-43: double, g-44: double, g-45: double, g-46: double, g-47: double, g-48: double, g-49: double, g-50: double, g-51: double, g-52: double, g-53: double, g-54: double, g-55: double, g-56: double, g-57: double, g-58: double, g-59: double, g-60: double, g-61: double, g-62: double, g-63: double, g-64: double, g-65: double, g-66: double, g-67: double, g-68: double, g-69: double, g-70: double, g-71: double, g-72: double, g-73: double, g-74: double, g-75: double, g-76: double, g-77: double, g-78: double, g-79: double, g-80: double, g-81: double, g-82: double, g-83: double, g-84: double, g-85: double, g-86: double, g-87: double, g-88: double, g-89: double, g-90: double, g-91: double, g-92: double, g-93: double, g-94: double, g-95: double, g-96: double, g-97: double, g-98: double, g-99: double, g-100: double, g-101: double, g-102: double, g-103: double, g-104: double, g-105: double, g-106: double, g-107: double, g-108: double, g-109: double, g-110: double, g-111: double, g-112: double, g-113: double, g-114: double, g-115: double, g-116: double, g-117: double, g-118: double, g-119: double, g-120: double, g-121: double, g-122: double, g-123: double, g-124: double, g-125: double, g-126: double, g-127: double, g-128: double, g-129: double, g-130: double, g-131: double, g-132: double, g-133: double, g-134: double, g-135: double, g-136: double, g-137: double, g-138: double, g-139: double, g-140: double, g-141: double, g-142: double, g-143: double, g-144: double, g-145: double, g-146: double, g-147: double, g-148: double, g-149: double, g-150: double, g-151: double, g-152: double, g-153: double, g-154: double, g-155: double, g-156: double, g-157: double, g-158: double, g-159: double, g-160: double, g-161: double, g-162: double, g-163: double, g-164: double, g-165: double, g-166: double, g-167: double, g-168: double, g-169: double, g-170: double, g-171: double, g-172: double, g-173: double, g-174: double, g-175: double, g-176: double, g-177: double, g-178: double, g-179: double, g-180: double, g-181: double, g-182: double, g-183: double, g-184: double, g-185: double, g-186: double, g-187: double, g-188: double, g-189: double, g-190: double, g-191: double, g-192: double, g-193: double, g-194: double, g-195: double, g-196: double, g-197: double, g-198: double, g-199: double, g-200: double, g-201: double, g-202: double, g-203: double, g-204: double, g-205: double, g-206: double, g-207: double, g-208: double, g-209: double, g-210: double, g-211: double, g-212: double, g-213: double, g-214: double, g-215: double, g-216: double, g-217: double, g-218: double, g-219: double, g-220: double, g-221: double, g-222: double, g-223: double, g-224: double, g-225: double, g-226: double, g-227: double, g-228: double, g-229: double, g-230: double, g-231: double, g-232: double, g-233: double, g-234: double, g-235: double, g-236: double, g-237: double, g-238: double, g-239: double, g-240: double, g-241: double, g-242: double, g-243: double, g-244: double, g-245: double, g-246: double, g-247: double, g-248: double, g-249: double, g-250: double, g-251: double, g-252: double, g-253: double, g-254: double, g-255: double, g-256: double, g-257: double, g-258: double, g-259: double, g-260: double, g-261: double, g-262: double, g-263: double, g-264: double, g-265: double, g-266: double, g-267: double, g-268: 

In [0]:
train_df.printSchema()

root
-- sig_id: string (nullable = true)
-- cp_type: string (nullable = true)
-- cp_time: integer (nullable = true)
-- cp_dose: string (nullable = true)
-- g-0: double (nullable = true)
-- g-1: double (nullable = true)
-- g-2: double (nullable = true)
-- g-3: double (nullable = true)
-- g-4: double (nullable = true)
-- g-5: double (nullable = true)
-- g-6: double (nullable = true)
-- g-7: double (nullable = true)
-- g-8: double (nullable = true)
-- g-9: double (nullable = true)
-- g-10: double (nullable = true)
-- g-11: double (nullable = true)
-- g-12: double (nullable = true)
-- g-13: double (nullable = true)
-- g-14: double (nullable = true)
-- g-15: double (nullable = true)
-- g-16: double (nullable = true)
-- g-17: double (nullable = true)
-- g-18: double (nullable = true)
-- g-19: double (nullable = true)
-- g-20: double (nullable = true)
-- g-21: double (nullable = true)
-- g-22: double (nullable = true)
-- g-23: double (nullable = true)
-- g-24: double (nullable = true)
-- g-25: double (nullable = true)
-- g-26: double (nullable = true)
-- g-27: double (nullable = true)
-- g-28: double (nullable = true)
-- g-29: double (nullable = true)
-- g-30: double (nullable = true)
-- g-31: double (nullable = true)
-- g-32: double (nullable = true)
-- g-33: double (nullable = true)
-- g-34: double (nullable = true)
-- g-35: double (nullable = true)
-- g-36: double (nullable = true)
-- g-37: double (nullable = true)
-- g-38: double (nullable = true)
-- g-39: double (nullable = true)
-- g-40: double (nullable = true)
-- g-41: double (nullable = true)
-- g-42: double (nullable = true)
-- g-43: double (nullable = true)
-- g-44: double (nullable = true)
-- g-45: double (nullable = true)
-- g-46: double (nullable = true)
-- g-47: double (nullable = true)
-- g-48: double (nullable = true)
-- g-49: double (nullable = true)
-- g-50: double (nullable = true)
-- g-51: double (nullable = true)
-- g-52: double (nullable = true)
-- g-53: double (nullable = true)
-- g-54: double (nullable = true)
-- g-55: double (nullable = true)
-- g-56: double (nullable = true)
-- g-57: double (nullable = true)
-- g-58: double (nullable = true)
-- g-59: double (nullable = true)
-- g-60: double (nullable = true)
-- g-61: double (nullable = true)
-- g-62: double (nullable = true)
-- g-63: double (nullable = true)
-- g-64: double (nullable = true)
-- g-65: double (nullable = true)
-- g-66: double (nullable = true)
-- g-67: double (nullable = true)
-- g-68: double (nullable = true)
-- g-69: double (nullable = true)
-- g-70: double (nullable = true)
-- g-71: double (nullable = true)
-- g-72: double (nullable = true)
-- g-73: double (nullable = true)
-- g-74: double (nullable = true)
-- g-75: double (nullable = true)
-- g-76: double (nullable = true)
-- g-77: double (nullable = true)
-- g-78: double (nullable = true)
-- g-79: double (nullable = true)
-- g-80: double (nullable = true)
-- g-81: double (nullable = true)
-- g-82: double (nullable = true)
-- g-83: double (nullable = true)
-- g-84: double (nullable = true)
-- g-85: double (nullable = true)
-- g-86: double (nullable = true)
-- g-87: double (nullable = true)
-- g-88: double (nullable = true)
-- g-89: double (nullable = true)
-- g-90: double (nullable = true)
-- g-91: double (nullable = true)
-- g-92: double (nullable = true)
-- g-93: double (nullable = true)
-- g-94: double (nullable = true)
-- g-95: double (nullable = true)
-- g-96: double (nullable = true)
-- g-97: double (nullable = true)
-- g-98: double (nullable = true)
-- g-99: double (nullable = true)
-- g-100: double (nullable = true)
-- g-101: double (nullable = true)
-- g-102: double (nullable = true)
-- g-103: double (nullable = true)
-- g-104: double (nullable = true)
-- g-105: double (nullable = true)
-- g-106: double (nullable = true)
-- g-107: double (nullable = true)
-- g-108: double (nullable = true)
-- g-109: double (nullable = true)
-- g-110: double (nullable = true)
-- g-111: double (nullable = true)
-- g-112: double (nullable = true)
--

In [0]:
target_df.printSchema()

root
-- sig_id: string (nullable = true)
-- 5-alpha_reductase_inhibitor: integer (nullable = true)
-- 11-beta-hsd1_inhibitor: integer (nullable = true)
-- acat_inhibitor: integer (nullable = true)
-- acetylcholine_receptor_agonist: integer (nullable = true)
-- acetylcholine_receptor_antagonist: integer (nullable = true)
-- acetylcholinesterase_inhibitor: integer (nullable = true)
-- adenosine_receptor_agonist: integer (nullable = true)
-- adenosine_receptor_antagonist: integer (nullable = true)
-- adenylyl_cyclase_activator: integer (nullable = true)
-- adrenergic_receptor_agonist: integer (nullable = true)
-- adrenergic_receptor_antagonist: integer (nullable = true)
-- akt_inhibitor: integer (nullable = true)
-- aldehyde_dehydrogenase_inhibitor: integer (nullable = true)
-- alk_inhibitor: integer (nullable = true)
-- ampk_activator: integer (nullable = true)
-- analgesic: integer (nullable = true)
-- androgen_receptor_agonist: integer (nullable = true)
-- androgen_receptor_antagonist: integer (nullable = true)
-- anesthetic_-_local: integer (nullable = true)
-- angiogenesis_inhibitor: integer (nullable = true)
-- angiotensin_receptor_antagonist: integer (nullable = true)
-- anti-inflammatory: integer (nullable = true)
-- antiarrhythmic: integer (nullable = true)
-- antibiotic: integer (nullable = true)
-- anticonvulsant: integer (nullable = true)
-- antifungal: integer (nullable = true)
-- antihistamine: integer (nullable = true)
-- antimalarial: integer (nullable = true)
-- antioxidant: integer (nullable = true)
-- antiprotozoal: integer (nullable = true)
-- antiviral: integer (nullable = true)
-- apoptosis_stimulant: integer (nullable = true)
-- aromatase_inhibitor: integer (nullable = true)
-- atm_kinase_inhibitor: integer (nullable = true)
-- atp-sensitive_potassium_channel_antagonist: integer (nullable = true)
-- atp_synthase_inhibitor: integer (nullable = true)
-- atpase_inhibitor: integer (nullable = true)
-- atr_kinase_inhibitor: integer (nullable = true)
-- aurora_kinase_inhibitor: integer (nullable = true)
-- autotaxin_inhibitor: integer (nullable = true)
-- bacterial_30s_ribosomal_subunit_inhibitor: integer (nullable = true)
-- bacterial_50s_ribosomal_subunit_inhibitor: integer (nullable = true)
-- bacterial_antifolate: integer (nullable = true)
-- bacterial_cell_wall_synthesis_inhibitor: integer (nullable = true)
-- bacterial_dna_gyrase_inhibitor: integer (nullable = true)
-- bacterial_dna_inhibitor: integer (nullable = true)
-- bacterial_membrane_integrity_inhibitor: integer (nullable = true)
-- bcl_inhibitor: integer (nullable = true)
-- bcr-abl_inhibitor: integer (nullable = true)
-- benzodiazepine_receptor_agonist: integer (nullable = true)
-- beta_amyloid_inhibitor: integer (nullable = true)
-- bromodomain_inhibitor: integer (nullable = true)
-- btk_inhibitor: integer (nullable = true)
-- calcineurin_inhibitor: integer (nullable = true)
-- calcium_channel_blocker: integer (nullable = true)
-- cannabinoid_receptor_agonist: integer (nullable = true)
-- cannabinoid_receptor_antagonist: integer (nullable = true)
-- carbonic_anhydrase_inhibitor: integer (nullable = true)
-- casein_kinase_inhibitor: integer (nullable = true)
-- caspase_activator: integer (nullable = true)
-- catechol_o_methyltransferase_inhibitor: integer (nullable = true)
-- cc_chemokine_receptor_antagonist: integer (nullable = true)
-- cck_receptor_antagonist: integer (nullable = true)
-- cdk_inhibitor: integer (nullable = true)
-- chelating_agent: integer (nullable = true)
-- chk_inhibitor: integer (nullable = true)
-- chloride_channel_blocker: integer (nullable = true)
-- cholesterol_inhibitor: integer (nullable = true)
-- cholinergic_receptor_antagonist: integer (nullable = true)
-- coagulation_factor_inhibitor: integer (nullable = true)
-- corticosteroid_agonist: integer (nullable = true)
-- cyclooxygenase_inhibitor: integer (nullable = true)
-- cytochrome_p450_inhibitor: integer (nullable = true)
-- dihydrofolate_reductase_inhibitor: integer

In [0]:
train_df.printSchema()

root
-- sig_id: string (nullable = true)
-- cp_type: string (nullable = true)
-- cp_time: integer (nullable = true)
-- cp_dose: string (nullable = true)
-- g-0: double (nullable = true)
-- g-1: double (nullable = true)
-- g-2: double (nullable = true)
-- g-3: double (nullable = true)
-- g-4: double (nullable = true)
-- g-5: double (nullable = true)
-- g-6: double (nullable = true)
-- g-7: double (nullable = true)
-- g-8: double (nullable = true)
-- g-9: double (nullable = true)
-- g-10: double (nullable = true)
-- g-11: double (nullable = true)
-- g-12: double (nullable = true)
-- g-13: double (nullable = true)
-- g-14: double (nullable = true)
-- g-15: double (nullable = true)
-- g-16: double (nullable = true)
-- g-17: double (nullable = true)
-- g-18: double (nullable = true)
-- g-19: double (nullable = true)
-- g-20: double (nullable = true)
-- g-21: double (nullable = true)
-- g-22: double (nullable = true)
-- g-23: double (nullable = true)
-- g-24: double (nullable = true)
-- g-25: double (nullable = true)
-- g-26: double (nullable = true)
-- g-27: double (nullable = true)
-- g-28: double (nullable = true)
-- g-29: double (nullable = true)
-- g-30: double (nullable = true)
-- g-31: double (nullable = true)
-- g-32: double (nullable = true)
-- g-33: double (nullable = true)
-- g-34: double (nullable = true)
-- g-35: double (nullable = true)
-- g-36: double (nullable = true)
-- g-37: double (nullable = true)
-- g-38: double (nullable = true)
-- g-39: double (nullable = true)
-- g-40: double (nullable = true)
-- g-41: double (nullable = true)
-- g-42: double (nullable = true)
-- g-43: double (nullable = true)
-- g-44: double (nullable = true)
-- g-45: double (nullable = true)
-- g-46: double (nullable = true)
-- g-47: double (nullable = true)
-- g-48: double (nullable = true)
-- g-49: double (nullable = true)
-- g-50: double (nullable = true)
-- g-51: double (nullable = true)
-- g-52: double (nullable = true)
-- g-53: double (nullable = true)
-- g-54: double (nullable = true)
-- g-55: double (nullable = true)
-- g-56: double (nullable = true)
-- g-57: double (nullable = true)
-- g-58: double (nullable = true)
-- g-59: double (nullable = true)
-- g-60: double (nullable = true)
-- g-61: double (nullable = true)
-- g-62: double (nullable = true)
-- g-63: double (nullable = true)
-- g-64: double (nullable = true)
-- g-65: double (nullable = true)
-- g-66: double (nullable = true)
-- g-67: double (nullable = true)
-- g-68: double (nullable = true)
-- g-69: double (nullable = true)
-- g-70: double (nullable = true)
-- g-71: double (nullable = true)
-- g-72: double (nullable = true)
-- g-73: double (nullable = true)
-- g-74: double (nullable = true)
-- g-75: double (nullable = true)
-- g-76: double (nullable = true)
-- g-77: double (nullable = true)
-- g-78: double (nullable = true)
-- g-79: double (nullable = true)
-- g-80: double (nullable = true)
-- g-81: double (nullable = true)
-- g-82: double (nullable = true)
-- g-83: double (nullable = true)
-- g-84: double (nullable = true)
-- g-85: double (nullable = true)
-- g-86: double (nullable = true)
-- g-87: double (nullable = true)
-- g-88: double (nullable = true)
-- g-89: double (nullable = true)
-- g-90: double (nullable = true)
-- g-91: double (nullable = true)
-- g-92: double (nullable = true)
-- g-93: double (nullable = true)
-- g-94: double (nullable = true)
-- g-95: double (nullable = true)
-- g-96: double (nullable = true)
-- g-97: double (nullable = true)
-- g-98: double (nullable = true)
-- g-99: double (nullable = true)
-- g-100: double (nullable = true)
-- g-101: double (nullable = true)
-- g-102: double (nullable = true)
-- g-103: double (nullable = true)
-- g-104: double (nullable = true)
-- g-105: double (nullable = true)
-- g-106: double (nullable = true)
-- g-107: double (nullable = true)
-- g-108: double (nullable = true)
-- g-109: double (nullable = true)
-- g-110: double (nullable = true)
-- g-111: double (nullable = true)
-- g-112: double (nullable = true)
--

In [0]:
def encode_cat_features(df, cat_features):

  indexed_cols = [''.join([col_name, '_indexed']) for col_name in cat_features]
  encoded_cols = [''.join([col_name, '_encoded']) for col_name in cat_features]
  string_indexers = [StringIndexer(inputCol=cat_features[i], outputCol=indexed_cols[i]) for i in range(len(cat_features))]
    
  encoder = OneHotEncoder(inputCols=indexed_cols, outputCols=encoded_cols)
  
  pipline = Pipeline(stages=string_indexers + [encoder])
  
  encoded_df = pipline.fit(df).transform(df)
  encoded_df = encoded_df.drop(*indexed_cols + cat_features)

  return encoded_df

def normalize_features(df, cols, normalizer, output_cols, if_drop=True):
  """
  """
  normalizer_lst = []
  vectorized_cols = []
  vector_assembers = []
  
  if isinstance(cols, list):
    cols = {'cols': cols}
  
  if isinstance(output_cols, str):
    output_cols = {'cols': output_cols}
  
  for k, v in cols.items():
    
    temp_normalizer = normalizer.copy()
    vectorized_col = ''.join([output_cols[k], '_v'])
    vector_assember = VectorAssembler(inputCols=v, outputCol=vectorized_col)
    
    temp_normalizer.setInputCol(vectorized_col)
    temp_normalizer.setOutputCol(output_cols[k])
    
    normalizer_lst.append(temp_normalizer)
    vectorized_cols.append(vectorized_col)
    vector_assembers.append(vector_assember)
  
  pipline = Pipeline(stages=vector_assembers + normalizer_lst)
  normalized_df = pipline.fit(df).transform(df).drop(*vectorized_cols)
  
  if if_drop:
    
    for k, v in cols.items():
      
      normalized_df = normalized_df.drop(v)
  
  return normalized_df

def add_pca_features(df, g_cols, c_cols, k=40):
  
  ## normalize g-col and c-col
  std_scaler = StandardScaler(withMean=True)
  
  input_cols = {
    'g_cols': g_cols, 
    'c_cols': c_cols}
  
  output_cols = {
    'g_cols': 'g_normalized', 
    'c_cols': 'c_normalized'}
  
  normalized_df = normalize_features(df, input_cols, std_scaler, output_cols, if_drop=False)
  
  ## perform PCA on g-cols and c-cols
  g_col_pca = PCA(k=k, inputCol='g_normalized', outputCol='g_col_pca')
  c_col_pca = PCA(k=k, inputCol='c_normalized', outputCol='c_col_pca')
  
  pipeline = Pipeline(stages=[g_col_pca, c_col_pca])
  pca_df = pipeline.fit(normalized_df).transform(normalized_df)
  
  return pca_df
  
def add_stats_features(df, g_cols, c_cols):
  
  @udf('double')
  def cols_sum(*lst):

    return sum(lst)

  @udf('double')
  def cols_mean(*lst):

    n = len(lst)
    s = sum(lst)

    return s / n

  @udf('double')
  def cols_var(*lst):

    n = len(lst)
    s = sum(lst) / n
    total = 0

    for x in lst:

      total += (x - s)**2 

    return total / n
  
  @udf('double')
  def cols_min(*lst):
    
    return min(lst)
  
  @udf('double')
  def cols_max(*lst):
    
    return max(lst)
  
  stats_dict = {
    'min_stats': cols_min,
    'max_stats': cols_max,
    'var_stats': cols_var,
    'mean_stats': cols_mean,
    'sum_stats': cols_sum
  }
  
  for name, func in stats_dict.items():
    
    df = df.withColumn(''.join(['g_cols_', name]), func(*[col(g_col) for g_col in g_cols]))
    df = df.withColumn(''.join(['c_cols_', name]), func(*[col(c_col) for c_col in c_cols]))
  
  return df

def add_kmeans_features(df, g_cols, c_cols, k=2, num_iter=10):
  
  kmeans_g = KMeans(k=k, featuresCol=g_cols, predictionCol='g_col_k_mean', seed=16)
  kmeans_c = KMeans(k=k, featuresCol=c_cols, predictionCol='c_col_k_mean', seed=16)
  
  kmeans_df = kmeans_g.fit(df).transform(df)
  kmeans_df = kmeans_c.fit(kmeans_df).transform(kmeans_df)
  
  return kmeans_df

def feature_engineering(df, num_cluster=2, num_comp=40, num_iter=10):
  
  ## get g-col and c-col
  g_cols = list(filter(lambda v: re.match('g-.+', v), df.columns))
  c_cols = list(filter(lambda v: re.match('c-.+', v), df.columns))
  
  ## PCA
  pca_df = add_pca_features(df, g_cols, c_cols, num_comp)

  ## stats features on g and c cols
  stats_df = add_stats_features(pca_df, g_cols, c_cols)
  
  ## add k-means features
  kmeans_df = add_kmeans_features(stats_df, g_cols='g_normalized', c_cols='c_normalized', k=num_cluster, num_iter=num_iter)
  
  return kmeans_df

In [0]:
## add indicator column to both train and test so we can combine them later
train_df = train_df.withColumn('is_test', lit(0))
test_df = test_df.withColumn('is_test', lit(1))

## Combine train and test df
full_df = train_df.union(test_df)

In [0]:
## encode features
target_cols = ['cp_type', 'cp_dose']
encoded_df = encode_cat_features(full_df, target_cols)

In [0]:
## feature engineering
fe_df = feature_engineering(encoded_df, num_comp=20, num_iter=5)

## select all the feature columns

pca_cols = list(filter(lambda v: re.match('.+_pca', v), fe_df.columns))
stats_cols = list(filter(lambda v: re.match('.+_stats', v), fe_df.columns))
k_means_cols = list(filter(lambda v: re.match('.+_k_mean', v), fe_df.columns))
cat_cols = list(filter(lambda v: re.match('.+_encoded', v), fe_df.columns)) + ['cp_time']

## stack them to a single feature vector
vector_assember_train = VectorAssembler(inputCols=pca_cols + stats_cols + k_means_cols + cat_cols, outputCol='all_features')
fe_df = vector_assember_train.transform(fe_df)

## normalize all the features
normalizer = StandardScaler(withMean=True)
cols = ['all_features']
output_cols = 'features'
fe_df = normalize_features(fe_df, cols, normalizer, output_cols, if_drop=False)

## split train, test df
fe_train = fe_df.filter(fe_df['is_test'] == 0)
final_test = fe_df.filter(fe_df['is_test'] == 1).select(['sig_id', 'features'])

## stack the training target
vector_assember_target = VectorAssembler(inputCols=[c for c in target_df.columns if c not in {'sig_id'}], outputCol='targets')
vectorized_target = vector_assember_target.transform(target_df).select(['sig_id', 'targets'])

## join training target with training features
final_train = fe_train.join(vectorized_target, ['sig_id']).select(['sig_id', 'features', 'targets'])

In [0]:
## train test split
(train, validation) = final_train.randomSplit([0.8, 0.2], 16)

train.cache()
validation.cache()

Out[11]: DataFrame[sig_id: string, features: vector, targets: vector]

In [0]:
class EnsembleBaseClfs:

    def __init__(self, base_clf, num_classes, feature_col, target_col):

        self.base_clf = base_clf
        self._trained_clfs = []
        self.num_classes = num_classes
        self.feature_col = feature_col
        self.target_col = target_col

    def fit(self, train):

        for c in range(num_classes):
          
            print(c)
            temp_clf = self.base_clf.copy()
            temp_train = train.withColumn('new_target', vector_to_array(self.target_col)[c])
            
            temp_clf.setLabelCol('new_target')
            temp_clf.setFeaturesCol(self.feature_col)
            
            temp_clf = temp_clf.fit(temp_train)
            self._trained_clfs.append(temp_clf)

        return self

    def predict(self, x_test):
      
      ## TODO
      
      pass

In [0]:
base_clf = LogisticRegression(maxIter=2)
num_classes = 5
feature_col = 'features'
target_col = 'targets'
clf = EnsembleBaseClfs(base_clf, num_classes, feature_col, target_col).fit(train)

0
1
2
3
4